In [ ]:
import cv2
import numpy as np
import os

# Create folders if not exists
os.makedirs("dataset/happy", exist_ok=True)
os.makedirs("dataset/sad", exist_ok=True)
os.makedirs("dataset/neutral", exist_ok=True)

# Load OpenCV's Haar cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
print("Press 'h' for Happy, 's' for Sad, 'n' for Neutral, 'q' to quit.")

count = {"happy": 0, "sad": 0, "neutral": 0}

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_crop = gray[y:y+h, x:x+w]
        face_crop = cv2.resize(face_crop, (64, 64))  # Resize for consistency
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('h'):
            cv2.imwrite(f"dataset/happy/happy_{count['happy']}.jpg", face_crop)
            count["happy"] += 1
        elif key == ord('s'):
            cv2.imwrite(f"dataset/sad/sad_{count['sad']}.jpg", face_crop)
            count["sad"] += 1
        elif key == ord('n'):
            cv2.imwrite(f"dataset/neutral/neutral_{count['neutral']}.jpg", face_crop)
            count["neutral"] += 1

    cv2.imshow("Face Data Collection", frame)

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Dataset collection complete. Run the training script next.")


Press 'h' for Happy, 's' for Sad, 'n' for Neutral, 'q' to quit.


NameError: name 'key' is not defined

: 

In [1]:
import cv2
import os
import numpy as np
import joblib
from skimage.feature import hog
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load images and labels
data = []
labels = []
emotions = ["happy", "sad", "neutral"]

for emotion in emotions:
    folder_path = f"dataset/{emotion}"
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Compute HOG features
        # It captures edge directions and gradients in an image, making it great for object detection, including face and emotion recognition.
        hog_features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        data.append(hog_features)
        labels.append(emotion)

# Convert to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Train an SVM classifier
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Save the trained model
joblib.dump((svm_model, label_encoder), "emotion_svm_hog.pkl")

print("Model trained and saved as 'emotion_svm_hog.pkl'. Now run the detection script!")


Model trained and saved as 'emotion_svm_hog.pkl'. Now run the detection script!


In [4]:
import cv2
import numpy as np
import joblib
from skimage.feature import hog

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load trained SVM model and label encoder
svm_model, label_encoder = joblib.load("emotion_svm_hog.pkl")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_crop = gray[y:y+h, x:x+w]
        face_crop = cv2.resize(face_crop, (64, 64))

        # Compute HOG features
        hog_features = hog(face_crop, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features = np.array(hog_features).reshape(1, -1)

        # Predict emotion

        prediction = svm_model.predict(hog_features)
        emotion = label_encoder.inverse_transform(prediction)[0]

        # Draw results
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('HOG-SVM Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()
